### DK Salaries Optimizer

In [8]:
import pandas as pd
import numpy as np
from pulp import *

In [9]:
def get_pos(name):
    l = name.split('\xa0')
    return l[-1] + '/UTIL'

'Giannis Antetokounmpo PF'.split('\xa0')

['Giannis Antetokounmpo', 'PF']

In [10]:
dk_data = pd.read_csv("numberfire.csv")
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To
0,Nikola Jokic C,53.5,"$9,900",5.40,38.08,23.3,10.8,8.0,1.4,0.8,3.7
1,Kawhi Leonard SF,50.8,"$10,300",4.93,37.57,27.7,7.7,5.6,2.0,0.7,3.1
2,LAC @ DENWed 9:00pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jayson Tatum SF/PF,48.2,"$9,300",5.18,39.66,26.1,9.0,3.6,1.6,1.0,2.9
4,TOR @ BOSWed 6:30pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# drop rows that have nan
dk_data = dk_data.dropna(axis=0, how='any')
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To
0,Nikola Jokic C,53.5,"$9,900",5.40,38.08,23.3,10.8,8.0,1.4,0.8,3.7
1,Kawhi Leonard SF,50.8,"$10,300",4.93,37.57,27.7,7.7,5.6,2.0,0.7,3.1
3,Jayson Tatum SF/PF,48.2,"$9,300",5.18,39.66,26.1,9.0,3.6,1.6,1.0,2.9
5,Paul George SG,44.0,"$8,500",5.18,37.37,22.7,6.9,5.0,1.9,0.5,3.3
7,Jamal Murray PG/SG,43.8,"$8,600",5.09,40.80,24.9,5.1,5.9,1.6,0.3,2.7


In [22]:
# Get the fields that are useful
#dk_data = dk_data[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame']]
dk_data['Roster Position'] = dk_data.apply(lambda x: get_pos(x['Player']),axis=1)

# change $s into int
dk_data['Cost'] = dk_data['Cost'].replace('[\$,]', '', regex=True).astype(float)

dk_data.head(10)

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To,Roster Position,PG,SG,SF,PF,C,G,F,UTIL
0,Nikola Jokic C,53.5,9900.0,5.40,38.08,23.3,10.8,8.0,1.4,0.8,3.7,Nikola Jokic C/UTIL,0,0,0,0,Nikola_Jokic_C_C,0,0,Nikola_Jokic_C_UTIL
1,Kawhi Leonard SF,50.8,10300.0,4.93,37.57,27.7,7.7,5.6,2.0,0.7,3.1,SF/UTIL,0,0,Kawhi_Leonard SF_SF,0,0,0,Kawhi_Leonard SF_F,Kawhi_Leonard SF_UTIL
3,Jayson Tatum SF/PF,48.2,9300.0,5.18,39.66,26.1,9.0,3.6,1.6,1.0,2.9,SF/PF/UTIL,0,0,Jayson_Tatum SF_PF_SF,Jayson_Tatum SF_PF_PF,0,0,Jayson_Tatum SF_PF_F,Jayson_Tatum SF_PF_UTIL
5,Paul George SG,44.0,8500.0,5.18,37.37,22.7,6.9,5.0,1.9,0.5,3.3,SG/UTIL,0,Paul_George SG_SG,0,0,0,Paul_George SG_G,0,Paul_George SG_UTIL
7,Jamal Murray PG/SG,43.8,8600.0,5.09,40.80,24.9,5.1,5.9,1.6,0.3,2.7,PG/SG/UTIL,Jamal_Murray PG_SG_PG,Jamal_Murray PG_SG_SG,0,0,0,Jamal_Murray PG_SG_G,0,Jamal_Murray PG_SG_UTIL
9,Kyle Lowry PG,41.1,8200.0,5.02,41.84,19.6,5.7,7.0,1.6,0.4,3.5,PG/UTIL,Kyle_Lowry PG_PG,0,0,0,0,Kyle_Lowry PG_G,0,Kyle_Lowry PG_UTIL
11,Pascal Siakam PF,41.1,7800.0,5.27,40.16,21.5,8.3,3.6,1.1,0.9,2.9,PF/UTIL,0,0,0,Pascal_Siakam PF_PF,0,0,Pascal_Siakam PF_F,Pascal_Siakam PF_UTIL
13,Kemba Walker PG,40.5,8100.0,5.00,38.29,22.2,4.9,5.4,1.1,0.6,2.6,PG/UTIL,Kemba_Walker PG_PG,0,0,0,0,Kemba_Walker PG_G,0,Kemba_Walker PG_UTIL
15,Fred VanVleet PG/SG,39.1,7500.0,5.22,41.53,18.2,4.1,7.3,2.0,0.3,2.5,PG/SG/UTIL,Fred_VanVleet PG_SG_PG,Fred_VanVleet PG_SG_SG,0,0,0,Fred_VanVleet PG_SG_G,0,Fred_VanVleet PG_SG_UTIL
17,Jaylen Brown SG/SF,35.6,7600.0,4.69,38.85,20.4,7.1,2.2,1.2,0.3,2.4,SG/SF/UTIL,0,Jaylen_Brown SG_SF_SG,Jaylen_Brown SG_SF_SF,0,0,Jaylen_Brown SG_SF_G,Jaylen_Brown SG_SF_F,Jaylen_Brown SG_SF_UTIL


In [23]:
injured_players = ['Gordon Hayward', 'Eric Bledsoe\xa0PG', 'test']

In [24]:
dk_data = dk_data[~dk_data['Player'].isin(injured_players)]

In [25]:
# Roster Positions for NBA 
nba_roster_positions = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UTIL']

SALARY_CAP = 50000
MAX_PLAYERS = 8


def make_vars(name, position, roster_position):
    if position in roster_position:
        return LpVariable(name + '_' + position,lowBound=0, cat='Binary')
    return 0

for position in nba_roster_positions:
    dk_data[position] = dk_data.apply(lambda x: make_vars(x['Player'],position, x['Roster Position']),axis=1)

    #[1 if position in x else 0 for x in dk_data['Roster Position']]

In [26]:
prob = LpProblem("OptimizeDK",LpMaximize)

In [27]:
names = dk_data['Player']
salaries = dk_data['Cost']
projected_points = dk_data['FP']

In [28]:
dk_data.head()

,Player,FP,Cost,Value,Min,Pts,Rbd,Ast,Stl,BLK,To,Roster Position,PG,SG,SF,PF,C,G,F,UTIL
0,Nikola Jokic C,53.5,9900.0,5.40,38.08,23.3,10.8,8.0,1.4,0.8,3.7,Nikola Jokic C/UTIL,0,0,0,0,Nikola_Jokic_C_C,0,0,Nikola_Jokic_C_UTIL
1,Kawhi Leonard SF,50.8,10300.0,4.93,37.57,27.7,7.7,5.6,2.0,0.7,3.1,SF/UTIL,0,0,Kawhi_Leonard SF_SF,0,0,0,Kawhi_Leonard SF_F,Kawhi_Leonard SF_UTIL
3,Jayson Tatum SF/PF,48.2,9300.0,5.18,39.66,26.1,9.0,3.6,1.6,1.0,2.9,SF/PF/UTIL,0,0,Jayson_Tatum SF_PF_SF,Jayson_Tatum SF_PF_PF,0,0,Jayson_Tatum SF_PF_F,Jayson_Tatum SF_PF_UTIL
5,Paul George SG,44.0,8500.0,5.18,37.37,22.7,6.9,5.0,1.9,0.5,3.3,SG/UTIL,0,Paul_George SG_SG,0,0,0,Paul_George SG_G,0,Paul_George SG_UTIL
7,Jamal Murray PG/SG,43.8,8600.0,5.09,40.80,24.9,5.1,5.9,1.6,0.3,2.7,PG/SG/UTIL,Jamal_Murray PG_SG_PG,Jamal_Murray PG_SG_SG,0,0,0,Jamal_Murray PG_SG_G,0,Jamal_Murray PG_SG_UTIL


In [29]:
# The objective function is added to 'prob' first
# OBJ FUNCTION
prob += lpSum([dk_data[i] * projected_points for i in nba_roster_positions]), 'Projected Points'

In [30]:
# CONSTRAINTS

# Stay under salary cap
prob += lpSum(salaries * dk_data[i] for i in nba_roster_positions) <= SALARY_CAP

# Stay under Num Players
prob += lpSum(dk_data[i] for i in nba_roster_positions) <= MAX_PLAYERS

# Select one of each position
for position in nba_roster_positions:
    prob += lpSum([dk_data[position]]) >= 1

# Don't use same player twice
for index, row in dk_data.iterrows():
    prob += lpSum(row[i] for i in nba_roster_positions) <= 1


In [31]:
# prob

In [32]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [33]:
prob.status

1

In [34]:
print("Total Estimated Points = ", value(prob.objective))


Total Estimated Points =  260.8


In [35]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue == 1:
        print(v.name)
        v.getName

Fred_VanVleet PG_SG_PG
Jayson_Tatum SF_PF_SF
Marc_Gasol C_C
Montrezl_Harrell C_UTIL
Pascal_Siakam PF_PF
Patrick_Beverley PG_G
Paul_George SG_SG
Paul_Millsap PF_C_F
